# Lab 7 code

In [175]:
import random
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf')
plt.rcParams['figure.figsize'] = (10,8)
import warnings
warnings.filterwarnings('ignore')
sns.set_context('notebook', font_scale=1.3)


def wta(items):
    maxweight = max(items)
    candidates = []
    for i in range(len(items)):
        if items[i] == maxweight:
            candidates.append(i)
    return random.choice(candidates)

def reception_weights(system, signal):
    weights = []
    for row in system:
        weights.append(row[signal])
    return weights

def communicate(speaker_system, hearer_system, meaning):
    speaker_signal = wta(speaker_system[meaning])
    hearer_meaning = wta(reception_weights(hearer_system, speaker_signal))
    if meaning == hearer_meaning: 
        return 1
    else: 
        return 0
    
def learn(system, meaning, signal, rule, rec_or_send):
    #print("in",system)
    for m in range(len(system[rec_or_send])):
        #print("meaning", m, meaning)
        for s in range(len(system[rec_or_send][m])):
            #print( "signal", s, signal)
            if m == meaning and s == signal: 
                system[rec_or_send][m][s] += rule[0]
            if m == meaning and s != signal: 
                system[rec_or_send][m][s] += rule[1]
            if m != meaning and s == signal: 
                system[rec_or_send][m][s] += rule[2]
            if m != meaning and s != signal: 
                system[rec_or_send][m][s] += rule[3]
    #print("out",system)

def pop_learn(population, data, no_learning_episodes, rule):
    for n in range(no_learning_episodes):
        ms_pair = random.choice(data)
        learn(random.choice(population), ms_pair[0][0], ms_pair[0][1], rule, 0) # learning the receiving system
        learn(random.choice(population), ms_pair[1][0], ms_pair[1][1], rule, 1) # learning the sending system
        
        
def pop_produce(population, no_productions):
    ms_pairs = []
    for n in range(no_productions):
        speaker = random.choice(population)
        meaning_receive = random.randrange(len(speaker[0]))
        meaning_send = random.randrange(len(speaker[1]))
        signal_receive = wta(speaker[0][meaning_receive])
        signal_send = wta(speaker[1][meaning_send])
        ms_pairs.append([[meaning_receive, signal_receive],[meaning_send, signal_send]])
    return ms_pairs

def ca_monte_pop(population, trials):
    total = 0.
    for n in range(trials):
        speaker = random.choice(population)
        hearer = random.choice(population)
        total += communicate(speaker, hearer, random.randrange(len(speaker)))
    return total / trials

def new_agent(initial_language_type):
    system = []
    for i in range(2):
        sig_rec = []
        for row_n in range(meanings):
            row = []
            for column_n in range(signals):
                if initial_language_type == 'optimal' and row_n == column_n:
                    row.append(1)
                else:
                    row.append(0)
            sig_rec.append(row)
        system.append(sig_rec)
    system.append([0., 0., 0., 0.])
    
    return system

def new_population(size, initial_language_type):
    population = []
    for i in range(size):
        population.append(new_agent(initial_language_type))
    return population

# Lab 4 code

In [176]:
import random
import pprint
import numpy as np
from copy import deepcopy
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf')
plt.rcParams['figure.figsize'] = (10,8)
import warnings
warnings.filterwarnings('ignore')
sns.set_context('notebook', font_scale=1.3)

def wta_bio(items):
    #print(items)
    maxweight = max(items)
    candidates = []
    for i in range(len(items)):
        if items[i] == maxweight:
            candidates.append(i)
    return random.choice(candidates)

def communicate_bio(speaker_system, hearer_system, meaning):
    #print(speaker_system)
    speaker_signal = wta_bio(speaker_system[meaning])
    hearer_meaning = wta_bio(hearer_system[speaker_signal])
    if meaning == hearer_meaning:
        return 1
    else:
        return 0
def communicate(speaker_system, hearer_system, meaning):
    speaker_signal = wta(speaker_system[meaning])
    hearer_meaning = wta(reception_weights(hearer_system, speaker_signal))
    if meaning == hearer_meaning: 
        return 1
    else: 
        return 0
    
    
def pop_update_bio(population):
    speaker_index = random.randrange(len(population))
    hearer_index = random.randrange(len(population) - 1)
    if hearer_index >= speaker_index: 
        hearer_index += 1
    speaker = population[speaker_index]
    #print("speaker in", speaker)
    hearer = population[hearer_index]
    #print("hearer in", hearer)
    meaning = random.randrange(len(speaker[0]))
    success = communicate_bio(speaker[0], hearer[1], meaning)
    speaker[2][0] += success
    speaker[2][1] += 1
    hearer[2][2] += success
    hearer[2][3] += 1
    #print("speaker out", speaker)
    #print("hearer out", hearer)
    
def fitness_bio(agent):
    send_success = agent[2][0]
    send_n = agent[2][1]
    receive_success = agent[2][2]
    receive_n = agent[2][3]
    if send_n == 0:
        send_n = 1
    if receive_n == 0:
        receive_n = 1
    return (((send_success/send_n) * send_weighting +
            (receive_success/receive_n) * receive_weighting) + 1
           )/(send_weighting + receive_weighting+1)

def sum_fitness_bio(population):
    total = 0
    for agent in population:
        total += fitness_bio(agent)
    return total

def mutate_bio(system):
    for row_i in range(len(system)):
        for column_i in range(len(system[0])):
            if random.random() < mutation_rate:
                system[row_i][column_i] = random.randint(0, mutation_max)
                
def pick_parent_bio(population, sum_f):
    accumulator = 0
    r = random.uniform(0, sum_f)
    for agent in population:
        accumulator += fitness_bio(agent)
        if r < accumulator:
            return agent

def new_population_bio(population):
    new_p = []
    sum_f = sum_fitness_bio(population)
    for i in range(len(population)):
        parent=pick_parent_bio(population, sum_f)
        child_production_system = deepcopy(parent[0])
        child_reception_system = deepcopy(parent[1])
        mutate_bio(child_production_system)
        mutate_bio(child_reception_system)
        child=[child_production_system,
               child_reception_system,
               [0., 0., 0., 0.]]
        new_p.append(child)
    return new_p

def random_system_bio(rows,columns):
    system = []
    for i in range(rows):
        row = []
        for j in range(columns):
            row.append(random.randint(0, mutation_max))
        system.append(row)
    return system

def random_population_bio(size):
    population = []
    for i in range(size):
        population.append([random_system(meanings, signals),
                           random_system(signals, meanings),
                           [0., 0., 0., 0.]])
    return population

def simulation_bio(generations):
    accumulator=[]
    population = random_population_bio(size)
    for i in range(generations):
        for j in range(interactions):
            pop_update_bio(population)
        average_fitness=(sum_fitness_bio(population) / size)
        accumulator.append(average_fitness)
        population = new_population_bio(population)
    return [population, accumulator]

# Combined + New code

In [188]:
mutation_rate = 0.01  # probability of mutation per weight
mutation_max = 1       # maximum value of a random weight
send_weighting = 10    # weighting factor for a send score
receive_weighting = 10 # weighting factor for receive score
meanings = 2                     # number of meanings
signals = 2                     # number of signals
interactions = 1000               # both the number of utterances produced and the number
                                 # of times this set is randomly sampled for training.
size = 100                      # size of population
method = 'evolution'             # method of population update
initial_language_type = 'random' # either 'optimal' or 'random'
           # learning rule (alpha, beta, gamma, delta)
    
def simulation(generations, mc_trials, report_every):
    population = new_population(size, initial_language_type)
    print(population)
    data_accumulator = []
    for i in range(generations + 1):
        #print i, # this line shows you the progress of the simulation. Delete if you find it annoying!
        #if (i % report_every == 0):
        #   data_accumulator.append(ca_monte_pop(population, mc_trials))
        data = pop_produce(population, interactions)
        #print(data)
        if method == 'chain':
            population = new_population(size, 'random')
            pop_learn(population, data, interactions, rule)
        if method == 'replacement':
            population = population[1:]
            learner = new_agent('random')
            pop_learn([learner], data, interactions, rule)
            population.append(learner)
        if method == 'closed':
            pop_learn(population, data, interactions, rule)
        if method == 'evolution':
            #
            for n in range(10):
                #print("generation {}".format(i), population)
                learner = population[random.randrange(len(population))]
                pop_learn([learner], data, interactions, rule)
                #print(population)
            for interaction in range(interactions):
                pop_update_bio(population)
            average_fitness=(sum_fitness_bio(population) / size)
            data_accumulator.append(average_fitness)
            #print(population)
            #data_accumulator.append(ca_monte_pop(population, mc_trials))
            population = new_population_bio(population)       
        #print(population)
    return [population, data_accumulator]

In [189]:


rules=[[[1,-1,-1,0],'#609693'], [[1,-1,-1,1],'#229693'], [[0,1,0,0],'#629333'], [[1,1,1,1],'#143333']]
for n in rules:
    rule = n[0]
    results=[]
    for i in range(1):
        
        print("run {}".format(i+1))
        pop, data = simulation(1000, 100, 10)
        results.append(data)
    graph = sns.tsplot(results, color='{}'.format(n[1]), alpha=0.7, condition='{}'.format(n[0])) 

figure_2=graph.set(xlabel='generations', ylabel='fitness')
title=plt.title('Figure 2: question one')

run 1
[[[[0, 0], [0, 0]], [[0, 0], [0, 0]], [0.0, 0.0, 0.0, 0.0]], [[[0, 0], [0, 0]], [[0, 0], [0, 0]], [0.0, 0.0, 0.0, 0.0]], [[[0, 0], [0, 0]], [[0, 0], [0, 0]], [0.0, 0.0, 0.0, 0.0]], [[[0, 0], [0, 0]], [[0, 0], [0, 0]], [0.0, 0.0, 0.0, 0.0]], [[[0, 0], [0, 0]], [[0, 0], [0, 0]], [0.0, 0.0, 0.0, 0.0]], [[[0, 0], [0, 0]], [[0, 0], [0, 0]], [0.0, 0.0, 0.0, 0.0]], [[[0, 0], [0, 0]], [[0, 0], [0, 0]], [0.0, 0.0, 0.0, 0.0]], [[[0, 0], [0, 0]], [[0, 0], [0, 0]], [0.0, 0.0, 0.0, 0.0]], [[[0, 0], [0, 0]], [[0, 0], [0, 0]], [0.0, 0.0, 0.0, 0.0]], [[[0, 0], [0, 0]], [[0, 0], [0, 0]], [0.0, 0.0, 0.0, 0.0]], [[[0, 0], [0, 0]], [[0, 0], [0, 0]], [0.0, 0.0, 0.0, 0.0]], [[[0, 0], [0, 0]], [[0, 0], [0, 0]], [0.0, 0.0, 0.0, 0.0]], [[[0, 0], [0, 0]], [[0, 0], [0, 0]], [0.0, 0.0, 0.0, 0.0]], [[[0, 0], [0, 0]], [[0, 0], [0, 0]], [0.0, 0.0, 0.0, 0.0]], [[[0, 0], [0, 0]], [[0, 0], [0, 0]], [0.0, 0.0, 0.0, 0.0]], [[[0, 0], [0, 0]], [[0, 0], [0, 0]], [0.0, 0.0, 0.0, 0.0]], [[[0, 0], [0, 0]], [[0, 0], [0, 0

KeyboardInterrupt: 